<a href="https://colab.research.google.com/github/FrustratedBoy420/Project-Exhibition/blob/main/Diabetic_Patient_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!git clone https://github.com/FrustratedBoy420/Project-Exhibition

fatal: destination path 'Project-Exhibition' already exists and is not an empty directory.


In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

In [33]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata # Corrected: Added this import

In [34]:


# Step 1: GPU availability check
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ----------------------------------
# 2. Device Setup
# ----------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Using device: {device}")

# ----------------------------------
# 3. Load Model & Tokenizer (with quantization fix)
# ----------------------------------
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
huggingface_token = userdata.get('HUGGING_FACE_TOKEN')  # Colab secret

# Quantization config (8-bit to save VRAM, use 4-bit if still OOM)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Changed to load_in_4bit=True
    bnb_4bit_quant_type="nf4", # Added 4-bit quantization type
    bnb_4bit_compute_dtype=torch.float16, # Added 4-bit compute dtype
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=True
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected use_auth_token to token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=bnb_config,
    token=huggingface_token # Corrected use_auth_token to token
)

print("✅ Mistral-7B model and tokenizer loaded successfully.")

Using device: cuda
✅ Using device: cuda


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [ ]:
import json

file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"Data se {len(chunks)} chunks mil gaye.")

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:


# Embedding model ko load karo
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Chunks ka embeddings banao
embeddings = embedding_model.encode(chunks, show_progress_bar=True)

print("Embeddings successfully created.")
print(f"Pehle chunk ka embedding shape: {embeddings[0].shape}")

In [ ]:
import numpy as np
import faiss

In [ ]:


# Embeddings ko float32 format mein convert karo, jo FAISS ke liye zaruri hai
embeddings_np = np.array(embeddings).astype('float32')
dimension = embeddings_np.shape[1]

# FAISS index banao (yeh ek index hai jo fast search mein madad karta hai)
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Index ko save karo taki dobara na banana pade
faiss.write_index(index, "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin")

print("FAISS index successfully created and saved.")

In [ ]:
# Yeh code tumhare existing code ke niche aayega
from google.colab import userdata

def get_recommendation_from_doc(user_query, chunks):
    # 1. User ke sawal ka embedding banao
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # 2. Vector database mein sabse relevant chunks dhoondo (top 3)
    k = 3
    distances, indices = index.search(query_embedding_np, k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # 3. Mistral-7B ke liye prompt banao
    prompt = f"""
    [INST]
    Based on the following medical guidelines from the American Diabetes Association, answer the user's question. Make sure your answer is helpful, accurate, and concise.

    Medical Guidelines: {context}

    User Question: {user_query}
    [/INST]
    """

    # 4. Model se jawab generate karvao
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=True,
            temperature=0.7,
            top_p=0.95
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# Ab isko test karo!
user_query = "What are the symptoms that a person have chances of having diabeties?"
recommendation = get_recommendation_from_doc(user_query, chunks)

print("\n--- Model's Recommendation ---")
print(f"Tumhara sawal: {user_query}")
print(f"Model ka jawab: {recommendation}")

##RAG TUNING


In [ ]:
!pip install -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu

In [ ]:
# ----------------------------------
# 1. Install Dependencies
# ----------------------------------

import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import json


In [ ]:
# # ----------------------------------
# # 2. Device Setup
# # ----------------------------------
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"✅ Using device: {device}")

# # ----------------------------------
# # 3. Load Model & Tokenizer (with quantization fix)
# # ----------------------------------
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
# huggingface_token = userdata.get('HUGGING_FACE_TOKEN')  # Colab secret

# # Quantization config (8-bit to save VRAM, use 4-bit if still OOM)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,  # Changed to load_in_4bit=True
#     bnb_4bit_quant_type="nf4", # Added 4-bit quantization type
#     bnb_4bit_compute_dtype=torch.float16, # Added 4-bit compute dtype
#     llm_int8_threshold=6.0,
#     llm_int8_has_fp16_weight=True
# )

# tokenizer = AutoTokenizer.from_pretrained(model_name, token=huggingface_token) # Corrected use_auth_token to token
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     device_map="auto",
#     quantization_config=bnb_config,
#     token=huggingface_token # Corrected use_auth_token to token
# )

# print("✅ Mistral-7B model and tokenizer loaded successfully.")

In [35]:
# ----------------------------------
# 4. Load Knowledge Base (JSON file)
# ----------------------------------
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            chunks.append(item['text'])

print(f"✅ Loaded {len(chunks)} chunks from knowledge base")

# ----------------------------------
# 5. Create / Load FAISS Index
# ----------------------------------
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for chunks
embeddings = embedding_model.encode(chunks, show_progress_bar=True)
embeddings_np = np.array(embeddings).astype('float32')

dimension = embeddings_np.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings_np)

# Save index
faiss.write_index(index, "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin")
print("✅ FAISS index successfully created and saved.")

✅ Loaded 638 chunks from knowledge base


Batches:   0%|          | 0/20 [00:00<?, ?it/s]

✅ FAISS index successfully created and saved.


In [ ]:
# ----------------------------------
# 6. RAG Function
# ----------------------------------
def get_recommendation_from_doc(user_query, chunks, embedding_model, index, model, tokenizer, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    # Determine device for the model - access device from a parameter
    device = next(model.parameters()).device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Convert tensors to FP16 if the model is in FP16, EXCEPT for input_ids
    if model.dtype == torch.float16:
         inputs = {k: (v.half() if k != 'input_ids' else v) for k, v in inputs.items()}


    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()
    return response_text

# ----------------------------------
# 7. Test Query
# ----------------------------------
# Assuming `loaded_chunks`, `loaded_embedding_model`, `loaded_index`,
# `loaded_model`, and `loaded_tokenizer` are available from previous cells.
# Make sure to run the cells that load these components before running this cell.

user_query = "What are the physical activity recommendations for a diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

In [36]:
# Save the model and tokenizer
# model_save_path = "/content/drive/MyDrive/LLM Model/mistral_rag_model"
# tokenizer_save_path = "/content/drive/MyDrive/LLM Model/mistral_rag_tokenizer"

# model.save_pretrained(model_save_path)
# tokenizer.save_pretrained(tokenizer_save_path)

# print(f"Model saved to: {model_save_path}")
# print(f"Tokenizer saved to: {tokenizer_save_path}")

# Example of how to load in another file:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# #
model_load_path = "/content/drive/MyDrive/LLM Model//mistral_rag_model"
tokenizer_load_path = "/content/drive/MyDrive/LLM Model//mistral_rag_tokenizer"

loaded_tokenizer = AutoTokenizer.from_pretrained(tokenizer_load_path)
loaded_model = AutoModelForCausalLM.from_pretrained(model_load_path)

print("Model and tokenizer loaded successfully in the new file.")

Model and tokenizer loaded successfully in the new file.


In [37]:
import torch
import numpy as np
import faiss
from transformers import AutoTokenizer, AutoModelForCausalLM
import json


print("✅ Model and tokenizer loaded successfully.")

# 1. Load the FAISS index
index_load_path = "/content/drive/MyDrive/LLM Model/diabetes_faiss_index.bin"
loaded_index = faiss.read_index(index_load_path)
print("✅ FAISS index loaded successfully.")

# 2. Load the chunks
file_path = "/content/Project-Exhibition/diabetes_knowledge_base (1).json"
loaded_chunks = []
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
    for item in data:
        if 'text' in item and item['text']:
            loaded_chunks.append(item['text'])
print(f"✅ Loaded {len(loaded_chunks)} chunks from knowledge base")

# 3. Load the embedding model
from sentence_transformers import SentenceTransformer
loaded_embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded successfully.")


# 4. Define the RAG Function using loaded components
def get_recommendation_from_doc(user_query, chunks, embedding_model, index, model, tokenizer, top_k=3):
    # Encode query
    query_embedding = embedding_model.encode([user_query])
    query_embedding_np = np.array(query_embedding).astype('float32')

    # Search FAISS
    distances, indices = index.search(query_embedding_np, top_k)
    relevant_chunks = [chunks[i] for i in indices[0]]
    context = " ".join(relevant_chunks)

    # Prompt
    prompt = f"""
    [INST]
    You are a medical assistant.
    Based on the following diabetes guidelines, answer the user's question clearly and concisely.

    Guidelines: {context}

    Question: {user_query}
    [/INST]
    """

    # Generate answer
    # Determine device for the model
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=350,
            do_sample=True,
            temperature=0.7,
            top_p=0.9
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("[/INST]")[-1].strip()

    return response_text

# 5. Test Query with loaded components
user_query = "What are the precautions for diabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

✅ Model and tokenizer loaded successfully.
✅ FAISS index loaded successfully.
✅ Loaded 638 chunks from knowledge base


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


✅ Embedding model loaded successfully.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.12 GiB. GPU 0 has a total capacity of 14.74 GiB of which 584.12 MiB is free. Process 159432 has 14.17 GiB memory in use. Of the allocated memory 13.19 GiB is allocated by PyTorch, and 872.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [38]:
# 5. Test Query with loaded components
user_query = "What are the precautions for prediabetic patient?"
recommendation = get_recommendation_from_doc(user_query, loaded_chunks, loaded_embedding_model, loaded_index, loaded_model, loaded_tokenizer)

print("\n--- Model's Recommendation (using loaded components) ---")
print(f"❓ Question: {user_query}")
print(f"✅ Answer: {recommendation}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.20 GiB. GPU 0 has a total capacity of 14.74 GiB of which 582.12 MiB is free. Process 159432 has 14.17 GiB memory in use. Of the allocated memory 13.29 GiB is allocated by PyTorch, and 772.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# =========================================
# PART 2 — QLoRA Fine-tuning + RAG Inference (OOM Safe)
# =========================================